<a href="https://colab.research.google.com/github/seongyongju/SAFE_AI_Project_Proposal/blob/main/Upload_CNN(Tuning_Ver)_NeurIPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torchvision import models, transforms
from torch.autograd import Variable
import torch.nn.functional as F

import matplotlib.pyplot as plt
import cv2
from google.colab import files

!pip install pdf2image
!apt-get install -y poppler-utils
from pdf2image import convert_from_path
from PIL import Image

In [ ]:
class CNN_FT(nn.Module):
    def __init__(self, p_dropout=0.3):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.bn1   = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2   = nn.BatchNorm2d(32)

        self.flat  = nn.Flatten()
        self.fc1   = nn.Linear(32*56*56, 128)
        self.dp1   = nn.Dropout(p_dropout)
        self.fc2   = nn.Linear(128, 2)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
            nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)

        x = self.flat(x)
        x = F.relu(self.fc1(x))
        x = self.dp1(x)
        x = self.fc2(x)
        return x

In [ ]:
def paper_to_image(pdf_file_name: str) -> str:
    images = convert_from_path(pdf_file_name, 200, fmt='jpg')
    num_page = len(images)

    if num_page < 7:
        raise Exception('최소 7페이지 이상')

    new_im = Image.new('RGB', (images[0].width * 4, images[0].height * 2), (255, 255, 255))

    for i in range(4):
        new_im.paste(images[i], (images[0].width * i, 0))

    for i in range(4, 8):
        new_im.paste(images[i], (images[0].width * (i - 4), images[0].height))

    img_file_name = pdf_file_name[:-4] + '.jpg'
    new_im = new_im.resize((3400, 2200))
    new_im.save(img_file_name)

    img = cv2.imread(img_file_name)
    img = cv2.resize(img, (680, 440))
    img[0:15, 0:150] = 255
    cv2.imwrite(img_file_name, img)

    return img_file_name

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN_FT().to(device)
model.load_state_dict(torch.load(
    '/content/drive/MyDrive/SAFE_AI(Project_proposal)/Models/NeurIPS_CNN_model_Tuning_VER.pt',
    map_location=device))
model.eval()

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

print("PDF 파일을 업로드하세요.")
uploaded = files.upload()
for fname in uploaded.keys():
    print(f"  • 업로드됨: {fname}")
    jpg = paper_to_image(fname)
    img = Image.open(jpg).convert('RGB')
    inp = preprocess(img).unsqueeze(0).to(device)
    with torch.no_grad():
        out = model(inp)
        prob = F.softmax(out,1)[0]
    print(f"  → Accepted: {prob[0].item()*100:.2f}%,  Rejected: {prob[1].item()*100:.2f}%")